In [87]:
import pandas as pd

In [88]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL to scrape
url = "https://www.pmindia.gov.in/en/details-of-foreigndomestic-visits/"
response = requests.get(url)
response.raise_for_status()

# Parse the HTML
soup = BeautifulSoup(response.text, "html.parser")
table = soup.find("table", class_="pms-list")

# Extract rows
rows = table.find_all("tr")
data = []

for tr in rows:
    cols = tr.find_all("td")
    if len(cols) >= 3:
        row = [td.get_text(strip=True) for td in cols[:3]]
        data.append(row)

# Create DataFrame
df = pd.DataFrame(data, columns=["sno", "place_of_visit", "period_of_visit"])
print(df.head())  # Show first few rows


  sno                                     place_of_visit  \
0   1                          United Kingdom & Maldives   
1   2  Ghana, Trinidad & Tobago, Argentina, Brazil & ...   
2   3                         Cyprus, Canada and Croatia   
3   4                                       Saudi Arabia   
4   5                               Thailand & Sri Lanka   

                period_of_visit  
0       23 July – 26 July, 2025  
1       02 July – 09 July, 2025  
2       15 June – 19 June, 2025  
3  22 April 2025 -22 April 2025  
4    3 April 2025 -6 April 2025  


In [89]:
df.columns

Index(['sno', 'place_of_visit', 'period_of_visit'], dtype='object')

In [90]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.pmindia.gov.in/en/previous-foreign-visits/"
response = requests.get(url)
response.raise_for_status()

soup = BeautifulSoup(response.text, "html.parser")
table = soup.find("table", class_="pms-list")

data = []
for tr in table.find_all("tr"):
    cols = tr.find_all("td")
    if len(cols) >= 4:
        row = [td.get_text(strip=True) for td in cols[:4]]
        data.append(row)

df_prev = pd.DataFrame(data, columns=["sno", "place_of_visit", "period_of_visit", "expenses"])
print(df_prev.head())


  sno place_of_visit                     period_of_visit      expenses
0   1         Brazil      13 November- 15 November, 2019  20,01,61,000
1   2       Thailand      02 November- 04 November, 2019   6,68,34,000
2   3   Saudi Arabia       28 October – 29 October, 2019   5,03,03,000
3   4            USA  21 September – 28 September , 2019  23,27,09,000
4   5         Russia  04 September – 05 September , 2019  12,02,80,000


In [91]:
df_prev.columns

Index(['sno', 'place_of_visit', 'period_of_visit', 'expenses'], dtype='object')

In [92]:
import pandas as pd
import numpy as np
import re

# Add 'expenses' column to df if missing
df['expenses'] = np.nan

# Concatenate both dataframes
df_combined = pd.concat([df, df_prev], ignore_index=True)

# List of known compound countries that contain '&' or 'and'
compound_countries = {
    'Trinidad & Tobago': '__TRINIDAD_TOBAGO__',
    'Bosnia & Herzegovina': '__BOSNIA_HERZEGOVINA__',
    'Antigua & Barbuda': '__ANTIGUA_BARBUDA__',
    'Saint Kitts & Nevis': '__SAINT_KITTS_NEVIS__',
    'Saint Vincent & The Grenadines': '__SAINT_VINCENT_GRENADINES__',
    'São Tomé & Príncipe': '__SAO_TOME_PRINCIPE__'
}

# Reverse mapping
reverse_map = {v: k for k, v in compound_countries.items()}

# Cleaning and splitting function
def clean_and_split_places(place):
    if pd.isnull(place):
        return []

    # Step 1: Lowercase
    place = place.lower()

    # Step 2: Replace known compound names with placeholders
    for name, placeholder in compound_countries.items():
        pattern = re.escape(name.lower())
        place = re.sub(pattern, placeholder, place)

    # Step 3: Replace separators with commas
    place = re.sub(r'\s+and\s+|\s*&\s*', ',', place)

    # Step 4: Split and strip
    places = [p.strip() for p in place.split(',') if p.strip()]

    # Step 5: Restore compound country names and title-case everything
    cleaned_places = [reverse_map.get(p, p).title() for p in places]

    return cleaned_places

# Apply the function
df_combined['place_list'] = df_combined['place_of_visit'].apply(clean_and_split_places)

# Explode the lists into rows
exploded_df = df_combined.explode('place_list')




In [93]:
exploded_df

,sno,place_of_visit,period_of_visit,expenses,place_list
0,1,United Kingdom & Maldives,"23 July – 26 July, 2025",NaN,United Kingdom
0,1,United Kingdom & Maldives,"23 July – 26 July, 2025",NaN,Maldives
1,2,"Ghana, Trinidad & Tobago, Argentina, Brazil & ...","02 July – 09 July, 2025",NaN,Ghana
1,2,"Ghana, Trinidad & Tobago, Argentina, Brazil & ...","02 July – 09 July, 2025",NaN,Trinidad & Tobago
1,2,"Ghana, Trinidad & Tobago, Argentina, Brazil & ...","02 July – 09 July, 2025",NaN,Argentina
...,...,...,...,...,...
87,54,USA,"25 Sept – 1 Oct, 2014","19,04,60,000",Usa
88,55,Japan,"30 Aug – 3 Sept, 2014","13,47,58,000",Japan
89,56,Nepal,"3 Aug – 5 Aug, 2014",IAF BBJ Aircraft,Nepal
90,57,Brazil,"13 July – 17 July, 2014","20,35,48,000",Brazil


In [94]:
# Drop empty or null entries
exploded_df = exploded_df[exploded_df['place_list'].notna() & (exploded_df['place_list'] != '')]

# Define a mapping of non-standard to standardized country names
country_rename_map = {
    'Usa': 'United States',
    'United States Of America (Usa)': 'United States',
    'Uae': 'United Arab Emirates',
    'United Arab Emirates (Uae)': 'United Arab Emirates',
    'Uk': 'United Kingdom',
    'Dubai': 'United Arab Emirates',
    'Samarkand': 'Uzbekistan',
}

# Apply the renaming to the 'place_list' column
exploded_df['place_list_standardized'] = exploded_df['place_list'].replace(country_rename_map)



In [95]:
exploded_df.place_list_standardized.unique()


array(['United Kingdom', 'Maldives', 'Ghana', 'Trinidad & Tobago',
       'Argentina', 'Brazil', 'Namibia', 'Cyprus', 'Canada', 'Croatia',
       'Saudi Arabia', 'Thailand', 'Sri Lanka', 'Mauritius', 'France',
       'United States', 'Kuwait', 'Nigeria', 'Guyana', 'Russia', 'Laos',
       'Brunei', 'Singapore', 'Poland', 'Ukraine', 'Austria', 'Italy',
       'Bhutan', 'United Arab Emirates', 'Qatar', 'Indonesia',
       'South Africa', 'Greece', 'Egypt', 'Japan', 'Papua New Guinea',
       'Australia', 'Uzbekistan', 'Germany', 'Nepal', 'Denmark',
       'Bangladesh', 'Kyrgyzstan', 'South Korea', 'Rwanda', 'Uganda',
       'China', 'Malaysia', 'Sweden', 'Jordan', 'Palestine', 'Oman',
       'Switzerland', 'Philippines', 'Myanmar', 'Israel', 'Portugal',
       'Netherlands', 'Kazakhstan', 'Spain', 'Vietnam', 'Mozambique',
       'Tanzania', 'Kenya', 'Afghanistan', 'Mexico', 'Iran', 'Belgium',
       'Pakistan', 'Turkey', 'Ireland', 'Turkmenistan', 'Tajikistan',
       'Mongolia', 'Seyche

In [ ]:
# exploded_df.to_csv('exploded_places.csv', index=False)

In [96]:
# Count frequency
place_counts = exploded_df['place_list_standardized'].value_counts()

# Convert to DataFrame
df_country_visits = place_counts.reset_index()

# Rename columns
df_country_visits.columns = ['country', 'count_visit']

# Show top 10
print(df_country_visits.head(10))


                country  count_visit
0         United States           10
1                 Japan            7
2                Russia            7
3               Germany            6
4  United Arab Emirates            6
5                France            6
6                 China            5
7                 Nepal            5
8             Singapore            5
9             Sri Lanka            4


In [98]:
df_country_visits.to_csv('country_visit_counts.csv', index=False)